# Normalisations

## Layer normalisation

Constat: Les entrées `X` des layers ne doivent être ni trop grandes ni trop petites, car sinon:

* La fonction à optimisée peut avoir de grandes hétérogénéités (phénomène du bol allongé)

* Les fonctions d'activations peuvent saturées = être évaluées uniquement là où elles sont constantes => pas de gradient => pas d'apprentissage

* Des gradients trop grand peuvent créer des nan


Solution: On doit normaliser:

* Mormaliser des inputs (on a l'habitude, par exemple on divise par 255 les images en `uint8`).

* Mais aussi, normaliser les tenseurs au coeur même du réseau.

La layer-normalisation est très simple: Supposons que `X` est le tenseur courant dans notre réseau de neurone:


    #X.shape = (batch_size, ..., n_feature)
    #ou ... peut être seq_len ou bien (height,width) ou rien

    mu = jnp.mean(X,axis=-1,keepdims=True)
    sigma = jnp.std(X,axis=-1,keepdims=True)
    X = (X-mu) / sigma

    #puis on dé-centre dénormalise de manière automatique:
    X = X*gamma + beta
    #ou gamma et beta sont apprenables


Cela fonctionne bien dès que `n_feature` est grand, ce qui est le cas dans les réseaux de neurones modernes. Mais attention, cette normalisation ne doit pas être appliqué à l'input (ex: pour une image n_feature = 3, pas assez pour estimer une moyenne et un écart type).

In [ ]:
!pip install equinox

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import jax
from jax import lax, vmap, jit
import jax.numpy as jnp
import jax.random as jr
import equinox as eqx
import jax.lax as lax


import time
import optax

import pickle
from dataclasses import dataclass
from typing import Callable
import os

In [ ]:
# La constante epsilon pour la stabilité numérique
_EPSILON = 1e-5


class LayerNormManual(eqx.Module):
    # Les paramètres apprenables
    gamma: jax.Array  # Paramètre de mise à l'échelle (scale)
    beta: jax.Array   # Paramètre de décalage (bias)

    # Non apprenable
    # L'axe sur lequel la normalisation est effectuée.
    # C'est généralement le dernier axe (axe des features/embed).
    axis: int = eqx.field(static=True)

    def __init__(self, normalized_dim: int, axis = -1, *, key):
        # Initialisation des paramètres apprenables (gamma et beta)
        # 1. 'gamma' est initialisé à 1.0 (ou un Array de ones)
        # 2. 'beta' est initialisé à 0.0 (ou un Array de zeros)
        # Ces paramètres sont de la taille de la dimension sur laquelle on normalise.
        self.gamma = jnp.ones(normalized_dim)
        self.beta = jnp.zeros(normalized_dim)

        # L'axe de normalisation
        self.axis = axis

    def __call__(self, x: jax.Array) -> jax.Array:
        """
        Applique la Layer Normalization à l'entrée `x`.

        Args:
            x: Le tenseur d'entrée (e.g., [Batch, SeqLen, EmbedDim]).
                La normalisation se fait sur la dimension spécifiée par 'self.axis'.

        Returns:
            Le tenseur normalisé (même forme que x).
        """

        # 1. Calculer la moyenne (mean) sur l'axe des features
        # L'argument `keepdims=True` est crucial pour que le tenseur 'mean'
        # puisse être soustrait de 'x' (Broadcasting).
        mean = jnp.mean(x, axis=self.axis, keepdims=True)

        # 2. Calculer la variance
        # La variance est (x - mean)^2 / N
        var = jnp.var(x, axis=self.axis, keepdims=True)

        # 3. Normalisation (standardisation)
        x_hat = (x - mean) / jnp.sqrt(var + _EPSILON)

        # 4. Mise à l'échelle et décalage (avec les paramètres apprenables gamma et beta)
        y = self.gamma * x_hat + self.beta

        return y

In [ ]:
import jax.random as jr

# 1. Paramètres
EMBED_DIM = 4  # La dimension des features/embeddings
BATCH_SIZE = 8
SEQ_LEN = 3
key = jr.key(42)

# 2. Création de l'entrée et du module
x_key, model_key = jr.split(key)
x = jr.normal(x_key, (BATCH_SIZE, SEQ_LEN, EMBED_DIM))

# Le module est initialisé avec la taille de la dimension à normaliser (EMBED_DIM)
ln_layer = LayerNormManual(normalized_dim=EMBED_DIM, key=model_key)

# 3. Exécution
y = ln_layer(x)

# 4. Vérification
# La moyenne et l'écart-type de la sortie 'y' doivent être proches de 0 et 1
# sur l'axe des features (le dernier axe, -1) pour chaque échantillon.
print(f"Shape de la sortie: {y.shape}")
print(f"Moyenne sur l'axe des features:\n {jnp.mean(y, axis=-1)}")
print(f"Écart-type sur l'axe des features:\n {jnp.std(y, axis=-1)}")

# Les valeurs doivent être très proches de 0.0 et 1.0.
# La moyenne de la sortie sur l'axe -1 sera proche de zeros,
# et l'écart-type sur l'axe -1 sera proche de ones, pour tous les (BATCH, SEQ_LEN).

## BatchNormalization

Supposons que `X` est le tenseur courant dans notre réseau de neurone.


    #X.shape = (batch_size, ..., n_feature)
    #ou ... peut être seq_len ou bien (height,width) ou rien

    mu ≈ jnp.mean(X,axis=0,keepdims=True)
    #mais le calcul de la moyenne lissé sur les batchs successifs
    
    sigma ≈ jnp.std(X,axis=0,keepdims=True)
    #idem
    
    X = (X-mu) / sigma

    #puis
    X = X*gamma + beta
    #ou gamma et beta sont apprenables


Attention, en mode 'inférence' (= pas 'train') on reprend la valeur du mu et sigma calculés pendant l'entrainement.

In [ ]:
import jax
import jax.numpy as jnp
import equinox as eqx

# Constantes
_EPSILON = 1e-5
_MOMENTUM = 0.9  # Facteur de lissage pour la moyenne/variance cumulée

class BatchNormManual(eqx.Module):
    # Paramètres Apprenables (jax.Array par défaut)
    gamma: jax.Array  # Facteur de mise à l'échelle
    beta: jax.Array   # Biais de décalage

    # État non-apprenable (doit être un jax.Array pour être mis à jour,
    # mais est marqué statique pour ne pas être optimisé)
    running_mean: jax.Array = eqx.field(static=True)
    running_variance: jax.Array = eqx.field(static=True)

    # Hyperparamètres statiques (simples entiers ou flottants)
    num_features: int = eqx.field(static=True)
    momentum: float = eqx.field(static=True)

    def __init__(self, num_features: int, momentum: float = _MOMENTUM, *, key: jax.random.PRNGKey):
        self.num_features = num_features
        self.momentum = momentum

        # 1. Initialisation des paramètres apprenables (taille = nombre de features/canaux)
        self.gamma = jnp.ones(num_features)
        self.beta = jnp.zeros(num_features)

        # 2. Initialisation de l'état cumulé (running stats)
        # Nécessite un jax.Array pour la mise à jour fonctionnelle
        self.running_mean = jnp.zeros(num_features)
        self.running_variance = jnp.ones(num_features)


    def __call__(self, x: jax.Array, mode_train: bool) -> tuple[jax.Array, 'BatchNormManual']:
        """
        Applique la Batch Normalization à l'entrée x.

        Args:
            x: Tenseur d'entrée (e.g., [Batch, H, W, Features/Channels]).
            state: Booléen indiquant si nous sommes en mode 'entraînement' (True)
                   ou 'inférence' (False).

        Returns:
            Tuple contenant :
                1. Le tenseur normalisé.
                2. Une nouvelle instance du module (l'état mis à jour si en mode entraînement).
        """

        # L'axe de normalisation est l'axe 0 (Batch), et tous les axes spatiaux.
        # Nous ne conservons que la dimension des features (le dernier axe par convention)

        # Axes sur lesquels nous calculons la moyenne et la variance (tous sauf le dernier)
        reduction_axes = tuple(range(x.ndim - 1))

        # --- Comportement Conditionnel ---
        if mode_train:
            # === 1. Mode Entraînement ===

            # Calculer la moyenne et la variance du Batch actuel
            batch_mean = jnp.mean(x, axis=reduction_axes, keepdims=False)
            batch_variance = jnp.var(x, axis=reduction_axes, keepdims=False)

            # Mise à jour des moyennes et variances cumulées
            # running_stat = momentum * running_stat + (1 - momentum) * batch_stat
            new_running_mean = self.momentum * self.running_mean + (1 - self.momentum) * batch_mean
            new_running_variance = self.momentum * self.running_variance + (1 - self.momentum) * batch_variance

            # Créer le nouveau module avec l'état mis à jour (Immuabilité JAX)
            new_module = eqx.tree_at(
                lambda m: (m.running_mean, m.running_variance),
                self,
                (new_running_mean, new_running_variance)
            )

            # Normalisation avec les statistiques du Batch
            x_hat = (x - batch_mean) / jnp.sqrt(batch_variance + _EPSILON)
            mean_to_use = batch_mean
            var_to_use = batch_variance

        else:
            # === 2. Mode Inférence ===

            # Le module ne change pas
            new_module = self

            # Normalisation avec les statistiques cumulées
            mean_to_use = self.running_mean
            var_to_use = self.running_variance

            # Pour la soustraction et la division, nous avons besoin que
            # les moyennes/variances aient la bonne forme pour le broadcasting.
            # Elles doivent avoir la même forme que `x`, sauf sur la dimension des features.
            mean_to_use = jnp.expand_dims(mean_to_use, axis=reduction_axes)
            var_to_use = jnp.expand_dims(var_to_use, axis=reduction_axes)

            x_hat = (x - mean_to_use) / jnp.sqrt(var_to_use + _EPSILON)

        # --- Application de Gamma et Beta ---
        # JAX gère le broadcasting de gamma et beta sur les axes du Batch et spatiaux
        y = self.gamma * x_hat + self.beta

        return y, new_module